In [1]:
import pandas as pd
import json

In [5]:
path = r"C:\Users\l1f22bscs0387\Desktop\movie_hit_predictor\data_cleaned\first_cleaned_dataset.csv"
clean_df = pd.read_csv(path)
clean_df.columns

Index(['Unnamed: 0', 'budget', 'popularity', 'vote_count',
       'top_cast_avg_rating', 'director_encoded', 'release_year',
       'release_month', 'Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie',
       'Thriller', 'War', 'Western', 'title', 'genres', 'director', 'top_cast',
       'release_date', 'vote_average', 'revenue'],
      dtype='object')

In [6]:
path_movies_metadata = r"C:\Users\l1f22bscs0387\Desktop\movie_hit_predictor\data_raw\movies_metadata.csv"
path_credits = r"C:\Users\l1f22bscs0387\Desktop\movie_hit_predictor\data_raw\credits.csv"

In [7]:
movies_metadata = pd.read_csv(path_movies_metadata)
credits = pd.read_csv(path_credits)

C:\Users\l1f22bscs0387\AppData\Local\Temp\ipykernel_33156\2751431668.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv(path_movies_metadata)


In [9]:
movies_metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [10]:
credits.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [11]:
movies_metadata['id'] = movies_metadata['id'].astype(str)
credits['id'] = credits['id'].astype(str)

In [13]:
merged_df = pd.merge(movies_metadata,credits,on='id',how='inner')
merged_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [15]:
columns_to_keep = [
    'title', 'budget', 'revenue', 'genres', 'release_date', 
    'cast', 'crew', 'popularity', 'vote_average', 'vote_count'
]

merged_df = merged_df[columns_to_keep]
merged_df.columns


Index(['title', 'budget', 'revenue', 'genres', 'release_date', 'cast', 'crew',
       'popularity', 'vote_average', 'vote_count'],
      dtype='object')

In [16]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45538 entries, 0 to 45537
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         45535 non-null  object 
 1   budget        45538 non-null  object 
 2   revenue       45535 non-null  float64
 3   genres        45538 non-null  object 
 4   release_date  45451 non-null  object 
 5   cast          45538 non-null  object 
 6   crew          45538 non-null  object 
 7   popularity    45535 non-null  object 
 8   vote_average  45535 non-null  float64
 9   vote_count    45535 non-null  float64
dtypes: float64(3), object(7)
memory usage: 3.5+ MB


In [24]:
clean_df2 = merged_df.copy()

In [25]:
numeric_cols = ['budget', 'revenue', 'popularity', 'vote_average', 'vote_count']

for col in numeric_cols:
    clean_df2[col] = pd.to_numeric(clean_df2[col])


In [31]:
clean_df2['release_date'] = pd.to_datetime(clean_df2['release_date'])
print(clean_df2['release_date'].dtype)

datetime64[ns]


In [32]:
clean_df2['release_year'] = clean_df2['release_date'].dt.year
clean_df2['release_month'] = clean_df2['release_date'].dt.month

In [38]:
clean_df2 = clean_df2.dropna(subset=['title','release_date'])

In [40]:
clean_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45451 entries, 0 to 45537
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          45451 non-null  object        
 1   budget         45451 non-null  int64         
 2   revenue        45451 non-null  float64       
 3   genres         45451 non-null  object        
 4   release_date   45451 non-null  datetime64[ns]
 5   cast           45451 non-null  object        
 6   crew           45451 non-null  object        
 7   popularity     45451 non-null  float64       
 8   vote_average   45451 non-null  float64       
 9   vote_count     45451 non-null  float64       
 10  release_year   45451 non-null  float64       
 11  release_month  45451 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(4)
memory usage: 4.5+ MB


In [41]:
clean_df.columns

Index(['Unnamed: 0', 'budget', 'popularity', 'vote_count',
       'top_cast_avg_rating', 'director_encoded', 'release_year',
       'release_month', 'Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie',
       'Thriller', 'War', 'Western', 'title', 'genres', 'director', 'top_cast',
       'release_date', 'vote_average', 'revenue'],
      dtype='object')

In [44]:
def parse_json_col(col):
    try:
        return json.loads(col.replace("'", '"'))
    except:
        return []


In [45]:
# Extract director
clean_df2['director'] = clean_df2['crew'].apply(
    lambda x: next((d['name'] for d in parse_json_col(x) if d.get('job') == 'Director'), None)
)

In [52]:
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

# Encode directors (reusing encoder from first dataset if available)
director_encoder = LabelEncoder()
clean_df2['director_encoded'] = director_encoder.fit_transform(clean_df2['director'])

In [70]:
import json, ast
from collections import defaultdict

# --- Step 1: Safer JSON parser ---
def parse_json_col_safe(col):
    try:
        return json.loads(col.replace("'", '"'))  # try normal JSON parse
    except:
        try:
            return ast.literal_eval(col)  # fallback to Python-style list/dict
        except:
            return []  # return empty if both fail

# --- Step 2: Extract top 3 cast members ---
clean_df2['top_cast'] = clean_df2['cast'].apply(
    lambda x: [d.get('name', None) for d in parse_json_col_safe(x)[:3] if isinstance(d, dict)]
)

# --- Step 3: Compute average rating per actor ---
actor_ratings2 = defaultdict(list)
for _, row in clean_df2.iterrows():
    for actor in row['top_cast']:
        if actor:  # skip None
            actor_ratings2[actor].append(row['vote_average'])

actor_avg_rating2 = {actor: sum(r) / len(r) for actor, r in actor_ratings2.items()}

# --- Step 4: Compute cast quality (average rating of top 3 actors) ---
def top_cast_avg_rating(top_cast):
    ratings = [actor_avg_rating2.get(actor, 0) for actor in top_cast]
    return sum(ratings) / len(ratings) if ratings else 0

clean_df2['top_cast_avg_rating'] = clean_df2['top_cast'].apply(top_cast_avg_rating)

# --- Step 5: Inspect results ---
print(clean_df2['top_cast_avg_rating'].describe())
clean_df2[['title', 'top_cast', 'top_cast_avg_rating']].head(10)


count    45451.000000
mean         5.379583
std          1.708624
min          0.000000
25%          5.155556
50%          5.806667
75%          6.247826
max         10.000000
Name: top_cast_avg_rating, dtype: float64


,title,top_cast,top_cast_avg_rating
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",6.056255
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",6.051724
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",6.164306
3,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devine]",5.641813
4,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short]",5.873770
5,Heat,"[Al Pacino, Robert De Niro, Val Kilmer]",6.255878
6,Sabrina,"[Harrison Ford, Julia Ormond, Greg Kinnear]",6.121617
7,Tom and Huck,"[Jonathan Taylor Thomas, Brad Renfro, Rachael ...",5.763147
8,Sudden Death,"[Jean-Claude Van Damme, Powers Boothe, Dorian ...",5.250724
9,GoldenEye,"[Pierce Brosnan, Sean Bean, Izabella Scorupco]",6.000185


In [71]:
clean_df2.columns

Index(['title', 'budget', 'revenue', 'genres', 'release_date', 'cast', 'crew',
       'popularity', 'vote_average', 'vote_count', 'release_year',
       'release_month', 'director', 'director_encoded', 'top_cast',
       'top_cast_avg_rating'],
      dtype='object')

In [77]:
import numpy as np
import pandas as pd
import ast
from collections import defaultdict
from sklearn.preprocessing import MultiLabelBinarizer

# -----------------------------
# 1️⃣ Log-transform numeric features
# -----------------------------
clean_df2['budget_log'] = np.log1p(clean_df2['budget'])
clean_df2['revenue_log'] = np.log1p(clean_df2['revenue'])

# -----------------------------
# 2️⃣ Profit ratio
# -----------------------------
clean_df2['profit_ratio'] = np.where(
    clean_df2['budget'] > 0,
    (clean_df2['revenue'] - clean_df2['budget']) / clean_df2['budget'],
    0
)

# -----------------------------
# 3️⃣ Parse and encode genres
# -----------------------------
def parse_genres(x):
    try:
        if isinstance(x, str):
            data = ast.literal_eval(x)
            return [d['name'] for d in data if isinstance(d, dict) and 'name' in d]
        return []
    except:
        return []

clean_df2['genres_list'] = clean_df2['genres'].apply(parse_genres)

mlb = MultiLabelBinarizer()
genre_dummies = pd.DataFrame(
    mlb.fit_transform(clean_df2['genres_list']),
    columns=mlb.classes_,
    index=clean_df2.index
)

clean_df2 = pd.concat([clean_df2, genre_dummies], axis=1)

# -----------------------------
# 4️⃣ Ensure director encoding is safe
# -----------------------------
clean_df2['director_encoded'] = clean_df2['director_encoded'].fillna(-1).astype(int)

# -----------------------------
# 5️⃣ Ensure cast rating column exists
# -----------------------------
if 'top_cast_avg_rating' not in clean_df2.columns:
    # In case not computed yet
    actor_ratings2 = defaultdict(list)
    for _, row in clean_df2.iterrows():
        for actor in [d['name'] for d in ast.literal_eval(row['cast'])[:3]]:
            actor_ratings2[actor].append(row['vote_average'])
    actor_avg_rating2 = {a: np.mean(r) for a, r in actor_ratings2.items()}

    def top_cast_avg_rating(top_cast):
        ratings = [actor_avg_rating2.get(actor, 0) for actor in top_cast]
        return np.mean(ratings) if ratings else 0

    clean_df2['top_cast_avg_rating'] = clean_df2['top_cast'].apply(top_cast_avg_rating)

# -----------------------------
# 6️⃣ Add release season
# -----------------------------
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

clean_df2['release_season'] = clean_df2['release_month'].apply(get_season)

# One-hot encode release season
season_dummies = pd.get_dummies(clean_df2['release_season'], prefix='season')
clean_df2 = pd.concat([clean_df2, season_dummies], axis=1)

# -----------------------------
# 7️⃣ Select final columns
# -----------------------------
final_cols = [
    'title', 'budget_log', 'revenue_log', 'profit_ratio', 'popularity',
    'vote_average', 'vote_count', 'director_encoded', 'top_cast_avg_rating',
    'release_year', 'release_month'
] + list(genre_dummies.columns) + list(season_dummies.columns)



In [78]:
# Convert all boolean columns (True/False) to 0/1 integers
for col in clean_df2.select_dtypes('bool').columns:
    clean_df2[col] = clean_df2[col].astype(int)


In [79]:

final_df2 = clean_df2[final_cols].copy()

# -----------------------------
# ✅ Preview
# -----------------------------
final_df2.head()

,title,budget_log,revenue_log,profit_ratio,popularity,vote_average,vote_count,director_encoded,top_cast_avg_rating,release_year,...,War,War,Western,Western,Western,Western,season_Winter,season_Winter,season_Winter,season_Winter
0,Toy Story,17.216708,19.738573,11.451801,21.946943,7.7,5415.0,2528,6.056255,1970.0,...,0,0,0,0,0,0,1,1,1,1
1,Jumanji,17.989898,19.386893,3.043035,17.015539,6.9,2413.0,2528,6.051724,1970.0,...,0,0,0,0,0,0,1,1,1,1
2,Grumpier Old Men,0.000000,0.000000,0.000000,11.712900,6.5,92.0,2528,6.164306,1970.0,...,0,0,0,0,0,0,1,1,1,1
3,Waiting to Exhale,16.588099,18.215526,4.090760,3.859495,6.1,34.0,2528,5.641813,1970.0,...,0,0,0,0,0,0,1,1,1,1
4,Father of the Bride Part II,0.000000,18.153832,0.000000,8.387519,5.7,173.0,2528,5.873770,1970.0,...,0,0,0,0,0,0,1,1,1,1


In [80]:
final_df2.columns

Index(['title', 'budget_log', 'revenue_log', 'profit_ratio', 'popularity',
       'vote_average', 'vote_count', 'director_encoded', 'top_cast_avg_rating',
       'release_year', 'release_month', 'Action', 'Action', 'Action', 'Action',
       'Adventure', 'Adventure', 'Adventure', 'Adventure', 'Animation',
       'Animation', 'Animation', 'Animation', 'Comedy', 'Comedy', 'Comedy',
       'Comedy', 'Crime', 'Crime', 'Crime', 'Crime', 'Documentary',
       'Documentary', 'Documentary', 'Documentary', 'Drama', 'Drama', 'Drama',
       'Drama', 'Family', 'Family', 'Family', 'Family', 'Fantasy', 'Fantasy',
       'Fantasy', 'Fantasy', 'Foreign', 'Foreign', 'Foreign', 'Foreign',
       'History', 'History', 'History', 'History', 'Horror', 'Horror',
       'Horror', 'Horror', 'Music', 'Music', 'Music', 'Music', 'Mystery',
       'Mystery', 'Mystery', 'Mystery', 'Romance', 'Romance', 'Romance',
       'Romance', 'Science Fiction', 'Science Fiction', 'Science Fiction',
       'Science Fiction', 

In [81]:
clean_df.columns

Index(['Unnamed: 0', 'budget', 'popularity', 'vote_count',
       'top_cast_avg_rating', 'director_encoded', 'release_year',
       'release_month', 'Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie',
       'Thriller', 'War', 'Western', 'title', 'genres', 'director', 'top_cast',
       'release_date', 'vote_average', 'revenue'],
      dtype='object')

In [82]:
final_df2.to_csv('second_cleaned_dataset.csv')